In [25]:
import chromadb
import pandas as pd
from sentence_transformers import SentenceTransformer

# 1️⃣ ChromaDB 클라이언트 설정 (로컬 DB 저장)
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# 2️⃣ ChromaDB 컬렉션 생성 (없으면 새로 만듦)
collection = chroma_client.get_or_create_collection(name="fashion_trends")

# 3️⃣ bge-ko 모델 로드 (한국어 특화 임베딩 모델)
embedding_model = SentenceTransformer("upskyy/bge-m3-korean")

# 4️⃣ CSV 데이터 로드
file_path = "2025-03-13_merged_fashion_trends.csv"  # 로컬에 저장된 CSV 파일 경로
df = pd.read_csv(file_path)

# 5️⃣ 기사 본문을 리스트로 변환
documents = df["본문"].tolist()
metadata = [{"title": t, "date": d, "url": u} for t, d, u in zip(df["제목"], df["업로드 날짜"], df["기사 URL"])]

# 6️⃣ 본문 임베딩 생성
embeddings = embedding_model.encode(documents, convert_to_numpy=True).tolist()

# 7️⃣ ChromaDB에 저장 (ID, 벡터, 메타데이터 포함)
for i, (embedding, doc) in enumerate(zip(embeddings, metadata)):
    collection.add(
        ids=[str(i)],
        embeddings=[embedding],
        metadatas=[doc]
    )

# ✅ 저장된 데이터 개수 확인
print(f"저장된 문서 개수: {collection.count()}")


저장된 문서 개수: 55


In [ ]:
## Structured Report 생성을 위한 RAG 시스템 구현 ##

import os
import chromadb
from google import genai
from sentence_transformers import SentenceTransformer
from langgraph.graph import StateGraph, END
from langchain.schema import Document
from dataclasses import dataclass, field
from typing import List
from dotenv import load_dotenv

# 🔹 1. .env 파일에서 API 키 불러오기
load_dotenv()  # ✅ .env 파일 로드
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")  # ✅ 환경 변수 가져오기

# 🔹 1-1. Google Gemini API 설정
client = genai.Client(api_key=GEMINI_API_KEY)  # ✅ .env에서 불러온 키 사용

# 🔹 2. ChromaDB 클라이언트 및 컬렉션 불러오기
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_collection(name="fashion_trends")

# 🔹 3. bge-m3-korean 임베딩 모델 로드
embedding_model = SentenceTransformer("upskyy/bge-m3-korean")

# 🔹 4. RAG 시스템의 상태 정의
@dataclass
class RAGState:
    query: str = ""
    documents: List[Document] = field(default_factory=list)
    response: str = ""  # ✅ 최종 Structured Output이 텍스트로 저장됨

# 🔹 5. 검색 노드 (ChromaDB에서 유사 문서 검색)
def search_node(state: RAGState) -> RAGState:
    query_embedding = embedding_model.encode([state.query], convert_to_numpy=True).tolist()
    results = collection.query(query_embeddings=query_embedding, n_results=3)

    docs = [
        Document(page_content=res["title"] + "\n" + res["url"], metadata={"date": res["date"]})
        for res in results["metadatas"][0]
    ]
    return RAGState(query=state.query, documents=docs, response="")

# 🔹 6. LLM 응답 생성 노드 (Structured Report 텍스트 생성)
def generate_node(state: RAGState) -> RAGState:
    context = "\n\n".join([doc.page_content for doc in state.documents])
    prompt = f"""
    당신은 패션 바잉 MD를 위한 트렌드 분석 AI입니다.
    다음 기사 내용을 참고하여 2025년 패션 트렌드를 분석하고 Structured Output 형식의 보고서를 작성하세요.

    🔹 예제 보고서 형식:
    
    📊 {state.query} 보고서
    
    ✅ 트렌드 요약
    [트렌드 요약 내용]

    ✅ 주요 트렌드
    1. [트렌드 이름]: [트렌드 설명]
    2. [트렌드 이름]: [트렌드 설명]
    3. [트렌드 이름]: [트렌드 설명]

    ✅ 바잉 추천
    📌 반드시 구매해야 할 아이템:
    - [아이템 1]
    - [아이템 2]
    - [아이템 3]

    📌 피해야 할 아이템:
    - [아이템 1]
    - [아이템 2]

    ✅ 추가 인사이트
    - [시장 분석 인사이트]
    - [소비자 행동 트렌드]

    🔹 기사 데이터:
    {context}

    🔹 위와 같은 형식으로 **Structured Output 형태의 보고서를 작성하세요.**
    """

    # ✅ Gemini API 호출 (텍스트 기반 Structured Report 생성)
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )

    return RAGState(query=state.query, documents=state.documents, response=response.text)

# 🔹 7. LangGraph Workflow 생성
workflow = StateGraph(RAGState)
workflow.add_node("search", search_node)
workflow.add_node("generate", generate_node)
workflow.set_entry_point("search")
workflow.add_edge("search", "generate")
workflow.add_edge("generate", END)
app = workflow.compile()

# 🔹 8. 실행 함수 (Structured Report 반환)
def run_rag(query: str):
    state = RAGState(query=query)
    result = app.invoke(state)

    # ✅ 최종 Structured Output 텍스트 출력
    return result.get("response", "❌ 보고서 생성 실패!")

# ✅ 실행 예시
query = "2025년 패션 바잉 전략을 위한 트렌드 분석"
report = run_rag(query)

# ✅ 최종 보고서 출력
print("\n📊 패션 바잉 전략 보고서:\n")
print(report)



📊 패션 바잉 전략 보고서:

## 📊 2025년 패션 바잉 전략을 위한 트렌드 분석 보고서

**✅ 트렌드 요약**

2025년 패션은 과거의 우아함과 현대적인 실용성이 결합된 스타일이 주를 이룰 것으로 예상됩니다. 특히 '네오-부르주아' 트렌드는 클래식한 아이템에 현대적인 감각을 더해 고급스러우면서도 편안한 스타일을 제시합니다. 또한, '레이어드 엘레강스'는 다양한 소재와 질감을 겹쳐 입어 깊이 있는 스타일을 연출하며, 파리 패션위크 스트리트 스타일에서 보여지듯 개성을 드러내는 과감한 시도와 믹스매치가 중요해질 것으로 보입니다.

**✅ 주요 트렌드**

1. **네오-부르주아 (Neo-Bourgeois):** 클래식한 아이템을 현대적으로 재해석하여 고급스러움과 편안함을 동시에 추구하는 트렌드입니다. 트위드 재킷, 플리츠 스커트, 셔츠 원피스 등 전통적인 아이템이 현대적인 디자인과 만나 새로운 스타일을 창조합니다. 컬러는 뉴트럴 톤을 중심으로 하며, 액세서리를 활용하여 포인트를 주는 것이 특징입니다.

2. **레이어드 엘레강스 (Layered Elegance):** 다양한 소재와 질감을 겹쳐 입어 깊이 있고 풍성한 스타일을 연출하는 트렌드입니다. 시폰, 실크, 니트 등 부드러운 소재를 레이어드하거나, 서로 다른 패턴과 컬러를 조합하여 개성을 드러낼 수 있습니다. 과감한 액세서리나 벨트를 활용하여 포인트를 주는 것도 좋은 방법입니다.

3. **믹스매치 & 개성 강조 (Mix & Match and Individualism):** 파리 패션위크 스트리트 스타일에서 나타나듯, 정형화된 스타일에서 벗어나 자유로운 믹스매치와 개성을 드러내는 스타일이 강조될 것입니다. 예상치 못한 소재와 컬러의 조합, 빈티지 아이템과 최신 트렌드의 조화 등을 통해 자신만의 독특한 스타일을 만들어내는 것이 중요합니다.

**✅ 바잉 추천**

📌 **반드시 구매해야 할 아이템:**

*   **트위드 재킷:** 네오-부르주아 트렌드를 대표하는 아이템으로, 다양한 컬러와 디자인으로 